In [7]:
data <- read_excel("Umempl_spatial.xlsx")

ERROR: Error: `path` does not exist: ‘Umempl_spatial.xlsx’


In [6]:
# 1. Создаем личную папку для библиотек, если её нет
dir.create(Sys.getenv("R_LIBS_USER"), recursive = TRUE, showWarnings = FALSE)

# 2. Говорим R использовать эту папку как основную
.libPaths(Sys.getenv("R_LIBS_USER"))

# 3. Указываем зеркало для скачивания
options(repos = c(CRAN = "https://cloud.r-project.org"))

# 4. Теперь устанавливаем пакеты (они полетят в твою папку)
install.packages(c("readxl", "dplyr", "spdep", "spatialreg", "ggplot2", "rgeoda"))

# 5. И отдельно sf (он самый тяжелый)
install.packages("sf")

Installing packages into ‘/home/vscode/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



also installing the dependencies ‘zoo’, ‘classInt’, ‘wk’, ‘proxy’, ‘mvtnorm’, ‘TH.data’, ‘sandwich’, ‘spData’, ‘sf’, ‘deldir’, ‘units’, ‘s2’, ‘e1071’, ‘sp’, ‘coda’, ‘LearnBayes’, ‘multcomp’, ‘Rcpp’, ‘BH’


Warning message in install.packages(c("readxl", "dplyr", "spdep", "spatialreg", :
“installation of package ‘s2’ had non-zero exit status”
Warning message in install.packages(c("readxl", "dplyr", "spdep", "spatialreg", :
“installation of package ‘sf’ had non-zero exit status”
Warning message in install.packages(c("readxl", "dplyr", "spdep", "spatialreg", :
“installation of package ‘spdep’ had non-zero exit status”
Warning message in install.packages(c("readxl", "dplyr", "spdep", "spatialreg", :
“installation of package ‘rgeoda’ had non-zero exit status”
Warning message in install.packages(c("readxl", "dplyr", "spdep", "spatialreg", :
“installation of package ‘spatialreg’ had non-zero exit status”
Installing package into ‘/home/vscode/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is uns

In [ ]:
library(readxl)      # Для чтения Экселя
library(dplyr)       # Для удобной работы с таблицами
library(spdep)       # Пространственная статистика
library(spatialreg)  # Регрессии (SAR, SEM)
library(ggplot2)     # Графики
library(rgeoda)      # Кластеры и веса

# Самый важный момент истины:
library(sf)          # Карты